In [ ]:
import requests
from bs4 import BeautifulSoup

base_urls = ['https://www.eriestclairhealthline.ca',
             'https://www.southwesthealthline.ca',
             'https://www.wwhealthline.ca',
             'https://www.hnhbhealthline.ca',
             'https://www.centralwesthealthline.ca',
             'https://www.mississaugahaltonhealthline.ca',
             'https://www.torontocentralhealthline.ca',
             'https://www.centralhealthline.ca',
             'https://www.centraleasthealthline.ca',
             'https://www.southeasthealthline.ca',
             'https://www.champlainhealthline.ca',
             'https://www.nsmhealthline.ca',
             'https://www.northeasthealthline.ca',
             'https://www.northwesthealthline.ca',
             ]

pharmacy_list = 'listServices.aspx?id=10345'

for base_url in base_urls:
    print(f'{base_url}{pharmacy_list}')
    page = requests.get(f'{base_url}/{pharmacy_list}')
    soup = BeautifulSoup(page.content, "html.parser")

    sub_urls = list(map(lambda x: x.find("a").get("href"), 
                       soup.find_all('td', class_='serviceListing')))
    for sub in sub_urls:
        page = requests.get(f'{base_url}/{sub}')
        soup = BeautifulSoup(page.content, "html.parser")
        name = soup.find('span', id='ctl00_ContentPlaceHolder1_lblProgram').text
        services = soup.find('span', id='ctl00_ContentPlaceHolder1_lblDescription').text
        
        try:
            phone = soup.find('span', id='ctl00_ContentPlaceHolder1_lblOfficePhone').text
            phone = re.sub(r'-', '', phone)
        except AttributeError as e:
            print(f"{name}: phone", e)
        
        try:
            fax = soup.find('span', id='ctl00_ContentPlaceHolder1_lblFax').text
            fax = re.sub(r'-', '', fax)
        except AttributeError as e:
            print(f"{name}: fax", e)
        
        try:
            fees = soup.find('span', id='ctl00_ContentPlaceHolder1_lblFees').text
        except AttributeError as e:
            print(f"{name}: fees", e)
        
        if 'phone' in globals():
            active_pharmacies.loc[active_pharmacies['Phone'] == phone, 'Services'] = services
            del services, phone
        elif 'fax' in globals():
            active_pharmacies.loc[active_pharmacies['Fax'] == fax, 'Services'] = services
            del services, fax    


In [ ]:
services = [
 'General pharmacy',
 'Compounding pharmacy',
 'compounding and mixing',
 'compounding',
 'medication placed in pill organizers (containers for storing scheduled doses) at no extra cost',
 'medication placed in blister packs (sealed, push-through packaging used',
 'treatment and prescription of medications for thirteen common medical ailments, including rashes, pink eye, insect bites, and urinary tract infections',
 'treatment and prescription of medications for thirteen common medical ailments, including rashes, pink eye, insect bites, and urinary tract',
 'home health care supplies',
 'basic home health care',
 'free seasonal flu',
 'geriatric pharmacist (BCGP) available for free consultation',
 'medication placed in pill organizers (containers for storing scheduled doses) at no extra',
 'medication placed in blister packs (sealed, push-through packing used to separate doses)',
 'free delivery for prescriptions',
 'MedsCheck - Appointments to review complex prescriptions with a pharmacist',
 'Flu vaccinations',
 'travel clinic',
 'free local delivery',
 'neighboring walk-in clinic',
 'compression stockings',
 'COVID-19 vaccines',
 'Safe disposal of prescription',
 'COVID-19 antiviral treatment (Paxlovid)',
 'Medications Return Program',
 'disease management and consultations',
 'MedsCheck -  ',
 'blood glucose monitor',
 'blood pressure',
 ]

active_pharmacies = pd.read_csv('./Active Pharmacies.csv')
total_service = ''

for row in active_pharmacies['Services']:
    if isinstance(row, str):
        total_service += row + ' '

total_service = re.sub(r'[\n\r]', '', total_service)
total_service = re.sub(r'[ \s\t]+', ' ', total_service)

for service in services:
    total_service = total_service.replace(service, '')
    
total_service = total_service.replace('free', '')
total_service = total_service.replace('basic', '')
total_service = total_service.replace('local', '')